<a href="https://colab.research.google.com/github/RxnAch/DeepLearning/blob/main/concise_implementation_of_linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U d2l

In [2]:
import numpy 
import torch
from torch.utils import data 
from d2l import torch as d2l

In [3]:
def synthetic_data(w,b,num_rows):
  X = torch.normal(0,1,(num_rows,len(w)))
  y = torch.matmul(X,w)+b #y = X.w + b
  y+= torch.normal(0,0.01,y.shape) #y = X.w+b+noises
  return X,y

In [4]:
true_w = torch.tensor([2,-3.4]) #Actual w while creating that y=X.w+b
true_b = 4.2 #Actual b
features,labels = d2l.synthetic_data(true_w,true_b,1000) 

 **Reading the dataset**

TensorDataset -> Dataset wrapping tensors.

DataLoader ->  It provides an iterable over the given dataset with optional automatic batching.

In [13]:
#example for TensorDataset and DataLoader
p = torch.tensor([[1,2,3],[4,5,6],[7,8,9],[41,5,62]])#inputs with 3 features(columns)
q = torch.tensor([2,5,8,3]) #labels
#create tensor dataset(simply dataset)
ds = data.TensorDataset(p,q)
for i in ds:
  print(i)
print("-----------------------------------------------")
ds_iter = data.DataLoader(ds,batch_size = 2,shuffle = True)
for i,data in enumerate(ds_iter):
  print(i,data)

(tensor([1, 2, 3]), tensor(2))
(tensor([4, 5, 6]), tensor(5))
(tensor([7, 8, 9]), tensor(8))
(tensor([41,  5, 62]), tensor(3))
-----------------------------------------------
0 [tensor([[4, 5, 6],
        [7, 8, 9]]), tensor([5, 8])]
1 [tensor([[ 1,  2,  3],
        [41,  5, 62]]), tensor([2, 3])]


In [6]:
def load_array(data_arrays,batch_size,is_train=True):
  dataset = data.TensorDataset(*data_arrays)
  return data.DataLoader(dataset,batch_size,shuffle = is_train)
batch_size = 10
data_iter = load_array((features,labels),batch_size)


In [7]:
from torch import nn
net = nn.Sequential(nn.Linear(2,1)) #Applies a linear transformation to the incoming data: y = X.w^T + b

In [8]:
net[0].weight.data.normal_(0,0.01) #initialize w for the 1st layer of NN.
net[0].bias.data.fill_(0) #initialize b for the first layer of NN.
print(f'w : {net[0].weight.data}, b: {net[0].bias.data}')

w : tensor([[-0.0061,  0.0017]]), b: tensor([0.])


The **MSELoss** class computes the mean squared error, also known as squared  L2  norm.
 By default it returns the average loss over examples.

In [9]:
loss = nn.MSELoss() 

In [10]:
for i in net.parameters():
  print(i)

Parameter containing:
tensor([[-0.0061,  0.0017]], requires_grad=True)
Parameter containing:
tensor([0.], requires_grad=True)


Defining the Optimization Algorithm

In [11]:
trainer = torch.optim.SGD(net.parameters(),lr=.03)  #lr = learning rate set to 0.03

In [12]:
num_epochs = 3
for epoch in range(num_epochs):
  for X,y in data_iter:
    l = loss(net(X),y)
    trainer.zero_grad() #set the gradients to zero before starting backpropagation
    # Else the gradient would point in some other direction than the intended direction
     #towards the minimum (or maximum, in case of maximization objectives).
    l.backward() #calculate gradients during backpropagation in loss wrt parameters
    trainer.step() #update the parameters
  l = loss(net(features),labels)
  print(f'epoch {epoch+1}, loss {l:f}')

epoch 1, loss 0.000206
epoch 2, loss 0.000100
epoch 3, loss 0.000101


In [14]:
w = net[0].weight.data
print('error in estimating w:', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('error in estimating b:', true_b - b)

error in estimating w: tensor([-0.0007, -0.0007])
error in estimating b: tensor([0.0001])
